In [28]:
import pandas as pd


file_path = "C:\\Users\\abubabu\\Documents\\GitHub\\synthetic-population-simulation\\NHTS Data Parser\\HH Data Parser\\trippub.csv"
# df = pd.read_csv(file_path)

columns_to_keep = ['HOUSEID', 'PERSONID',"STRTTIME","ENDTIME",'TRVLCMIN', 'R_SEX', 'R_AGE', 'WRKCOUNT','HHVEHCNT', 'HHSIZE','EDUC','WHYTRP1S','HHFAMINC']
final_data = pd.read_csv(file_path,usecols=columns_to_keep)




In [29]:
def translateAge(age):
    if age <=5:
        return 1
    elif age <= 18:
        return 2
    elif age <= 25:
        return 3
    elif age < 60:
        return 4
    else:
        return 5

# get the symbolic name of the gender from numerical value
def translateSex(sex):
    if sex == 1:
        return 'male'
    else:
        return 'female'

# get the race categories from numerical race values
def translateRace(race):
    if race == 1:
        return 1 #white
    elif race == 2:
        return 2 #black
    elif race == 4 and race ==6:
        return 3 #native
    elif race == 3:
        return 6 #asian
    elif race == 5:
        return 7 #'pacific-islander'
    else:
        return 8 #'other'
    
def translateInc(inc):
    if inc>=10: return 10
    if inc<1: return 0
    return inc   

In [30]:

final_data['R_AGE'] = final_data.apply(lambda row: translateAge(row['R_AGE']), axis=1)
# final_data['R_RACE'] = final_data.apply(lambda row: translateRace(row['R_RACE']), axis=1)
final_data['HHFAMINC'] = final_data.apply(lambda row: translateInc(row['HHFAMINC']), axis=1)


In [31]:
final_data.head(10)

,HOUSEID,PERSONID,STRTTIME,ENDTIME,TRVLCMIN,WHYTRP1S,HHSIZE,HHVEHCNT,HHFAMINC,WRKCOUNT,R_AGE,EDUC,R_SEX
0,30000007,1,1000,1015,15,20,3,5,7,1,5,3,2
1,30000007,1,1510,1530,20,1,3,5,7,1,5,3,2
2,30000007,2,700,900,120,1,3,5,7,1,5,3,1
3,30000007,2,1800,2030,150,10,3,5,7,1,5,3,1
4,30000007,3,845,900,15,20,3,5,7,1,4,2,2
5,30000007,3,1430,1445,15,1,3,5,7,1,4,2,2
6,30000008,1,1115,1130,15,10,2,4,8,2,4,5,1
7,30000008,1,2330,2340,10,1,2,4,8,2,4,5,1
8,30000012,1,550,605,15,50,1,2,10,1,4,5,2
9,30000012,1,700,715,15,1,1,2,10,1,4,5,2


In [32]:
final_data['HHFAMINC'].value_counts()

6     160854
10    138405
7     132897
8     110961
5     100756
4      70939
9      64388
3      59997
2      30177
1      29858
0      24340
Name: HHFAMINC, dtype: int64

In [49]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler

# Assuming "final_data" is a pandas DataFrame containing your dataset
# Make sure you have already preprocessed the data and removed any missing values.

# Extracting the features (X) and target (y) from the dataset
# X = final_data.drop(columns=['TRIPPURP','WHYFROM','HOUSEID', 'PERSONID',"TDAYDATE",'WRKCOUNT'],axis=1)
X = final_data.drop(columns=['WHYTRP1S','HOUSEID', 'PERSONID'],axis=1)
y = final_data['WHYTRP1S']

# Preprocess 'R_AGE' and 'R_SEX' using LabelEncoder
label_encoder = LabelEncoder()
X['R_AGE'] = label_encoder.fit_transform(X['R_AGE'])
X['R_SEX'] = label_encoder.fit_transform(X['R_SEX'])


# Preprocess the target variable using LabelEncoder
y_label_encoder = LabelEncoder()
y_encoded = y_label_encoder.fit_transform(y)

# Convert the target variable to one-hot encoded format
num_classes = len(y_label_encoder.classes_)
y_one_hot = pd.get_dummies(y_encoded)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

# X_train=X.astype('float64')
num_classes
# Standardize the features (optional, but often recommended for neural networks)
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
input_shape = X_train.shape[1]

# X_test_scaled.shape
print(X_train_scaled.shape)
print(y_train.shape)
print(X.shape)
print(y.shape)

(738857, 10)
(738857, 9)
(923572, 10)
(923572,)


In [43]:
y_one_hot.value_counts()

0  1  2  3  4  5  6  7  8
1  0  0  0  0  0  0  0  0    318777
0  0  0  0  1  0  0  0  0    184126
   1  0  0  0  0  0  0  0    110590
   0  0  0  0  1  0  0  0    100284
               0  0  1  0     72327
                  1  0  0     56377
      1  0  0  0  0  0  0     43397
      0  0  0  0  0  0  1     20910
         1  0  0  0  0  0     16784
dtype: int64

In [ ]:
X_train_scaled

array([[0.36286562, 0.3819415 , 0.01157982, ..., 0.16666667, 0.25      ,
        0.76923077],
       [0.69944892, 0.70156846, 0.01157982, ..., 0.08333333, 0.08333333,
        0.92307692],
       [0.50869012, 0.51292921, 0.01571547, ..., 0.08333333, 0.41666667,
        0.53846154],
       ...,
       [0.29673591, 0.29885545, 0.01157982, ..., 0.16666667, 0.08333333,
        0.76923077],
       [0.86689275, 0.86901229, 0.01157982, ..., 0.16666667, 0.25      ,
        1.        ],
       [0.89020772, 0.89656634, 0.01985112, ..., 0.33333333, 0.16666667,
        0.92307692]])

In [39]:


# Create the neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))
# model = Sequential()

# # Add layers to the model
# model.add(Dense(64, activation='relu', input_shape=(input_shape,)))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(1))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train_scaled, y_train, epochs=3, batch_size=32, validation_data=(X_test_scaled, y_test))
# model.fit(X_train, y, epochs=3, batch_size=32, validation_split=0.2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print("Test loss:", loss)
print("Test accuracy:", accuracy)

Epoch 1/3


ValueError: in user code:

    File "C:\Users\abubabu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\abubabu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\abubabu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\abubabu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\keras\engine\training.py", line 1024, i

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                576       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 6)                 198       
                                                                 
Total params: 2,854
Trainable params: 2,854
Non-trainable params: 0
_________________________________________________________________


In [ ]:
X_train_scaled.shape

(923572, 8)

In [ ]:
model.save("C:\\Users\\abubabu\\Documents\\GitHub\\synthetic-population-simulation\\Model\\v1.h5")

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# Assuming you have X_train, X_test, y_train, y_test from the train-test split

# Create the Decision Tree classifier
dt_classifier = DecisionTreeClassifier(random_state=42)

# Train the model on the training data
dt_classifier.fit(X_train_scaled, y_train)

# Make predictions on the test data
y_pred = dt_classifier.predict(X_test_scaled)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.25870665620009203
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.36      0.26      0.30     38092
           2       0.29      0.19      0.23     38881
           3       0.23      0.15      0.18     21993
           4       0.36      0.28      0.31     23445
           5       0.43      0.33      0.37     62299

   micro avg       0.36      0.26      0.30    184715
   macro avg       0.28      0.20      0.23    184715
weighted avg       0.35      0.26      0.30    184715
 samples avg       0.26      0.26      0.26    184715



C:\Users\abubabu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_classification.py:1327: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [46]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau

# Assuming "final_data" is a pandas DataFrame containing your dataset
# Make sure you have already preprocessed the data and removed any missing values.

# Extracting the features (X) and target (y) from the dataset
# X = final_data.drop(columns=['TRIPPURP', 'WHYFROM'])
# y = final_data['WHYFROM']

# # Preprocess 'R_AGE' and 'R_SEX' using LabelEncoder
# label_encoder = LabelEncoder()
# X['R_AGE'] = label_encoder.fit_transform(X['R_AGE'])
# X['R_SEX'] = label_encoder.fit_transform(X['R_SEX'])

# # Preprocess the target variable using LabelEncoder
# y_label_encoder = LabelEncoder()
# y_encoded = y_label_encoder.fit_transform(y)

# # Convert the target variable to one-hot encoded format
# num_classes = len(y_label_encoder.classes_)
# y_one_hot = pd.get_dummies(y_encoded)

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

# # Standardize the features (optional, but often recommended for neural networks)
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# Create the neural network model
model = Sequential()
model.add(Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dropout(0.5))  # Adding dropout for regularization
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))  # Adding dropout for regularization
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
optimizer = Adam(learning_rate=0.001)  # You can experiment with different learning rates
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Adding learning rate scheduler to reduce learning rate during training
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

# Train the model
model.fit(X_train_scaled, y_train, epochs=5, batch_size=32, validation_data=(X_test_scaled, y_test), callbacks=[reduce_lr])

# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print("Test loss:", loss)
print("Test accuracy:", accuracy)

Epoch 1/5
23090/23090 [==============================] - 24s 1ms/step - loss: 1.6864 - accuracy: 0.4146 - val_loss: 1.6198 - val_accuracy: 0.4348 - lr: 0.0010
Epoch 2/5
23090/23090 [==============================] - 24s 1ms/step - loss: 1.6571 - accuracy: 0.4230 - val_loss: 1.6073 - val_accuracy: 0.4415 - lr: 0.0010
Epoch 3/5
23090/23090 [==============================] - 24s 1ms/step - loss: 1.6502 - accuracy: 0.4253 - val_loss: 1.6033 - val_accuracy: 0.4377 - lr: 0.0010
Epoch 4/5
23090/23090 [==============================] - 24s 1ms/step - loss: 1.6450 - accuracy: 0.4261 - val_loss: 1.6025 - val_accuracy: 0.4406 - lr: 0.0010
Epoch 5/5
5773/5773 [==============================] - 3s 603us/step - loss: 1.5954 - accuracy: 0.4421
Test loss: 1.5953700542449951
Test accuracy: 0.44205397367477417


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error, r2_score
model = LogisticRegression()
model.fit(X_train_scaled, y_train)
# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

ValueError: y should be a 1d array, got an array of shape (738857, 6) instead.

In [47]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler

# # Assuming "final_data" is a pandas DataFrame containing your dataset
# # Make sure you have already preprocessed the data and removed any missing values.

# # Extracting the features (X) and target (y) from the dataset
# X = final_data.drop(columns=['TRIPPURP', 'WHYFROM'])
# y = final_data['WHYFROM']

# # Preprocess 'R_AGE' and 'R_SEX' using LabelEncoder
# label_encoder = LabelEncoder()
# X['R_AGE'] = label_encoder.fit_transform(X['R_AGE'])
# X['R_SEX'] = label_encoder.fit_transform(X['R_SEX'])

# # Preprocess the target variable using LabelEncoder
# y_label_encoder = LabelEncoder()
# y_encoded = y_label_encoder.fit_transform(y)

# # Convert the target variable to one-hot encoded format
# num_classes = len(y_label_encoder.classes_)
# y_one_hot = pd.get_dummies(y_encoded)

# # Split the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y_one_hot, test_size=0.2, random_state=42)

# # Standardize the features (optional, but often recommended for neural networks)
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# Create the neural network model
model = Sequential()
model.add(Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(BatchNormalization())  # Batch Normalization
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())  # Batch Normalization
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Define a learning rate scheduler
def lr_scheduler(epoch):
    return 0.001 * np.exp(-epoch / 10)

# Compile the model with SGD optimizer and learning rate scheduler
optimizer = SGD(learning_rate=0.0)  # Use a placeholder learning rate, the scheduler will update it
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Set up learning rate scheduler
lr_scheduler_callback = LearningRateScheduler(lr_scheduler)

# Train the model
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_data=(X_test_scaled, y_test), callbacks=[lr_scheduler_callback])

# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print("Test loss:", loss)
print("Test accuracy:", accuracy)

Epoch 1/10
23090/23090 [==============================] - 28s 1ms/step - loss: 1.8700 - accuracy: 0.3617 - val_loss: 1.6723 - val_accuracy: 0.4276 - lr: 0.0010
Epoch 2/10
23090/23090 [==============================] - 28s 1ms/step - loss: 1.7341 - accuracy: 0.3992 - val_loss: 1.6517 - val_accuracy: 0.4265 - lr: 9.0484e-04
Epoch 3/10
23090/23090 [==============================] - 28s 1ms/step - loss: 1.7137 - accuracy: 0.4037 - val_loss: 1.6431 - val_accuracy: 0.4260 - lr: 8.1873e-04
Epoch 4/10
23090/23090 [==============================] - 27s 1ms/step - loss: 1.7028 - accuracy: 0.4066 - val_loss: 1.6377 - val_accuracy: 0.4284 - lr: 7.4082e-04
Epoch 5/10
23090/23090 [==============================] - 27s 1ms/step - loss: 1.6950 - accuracy: 0.4094 - val_loss: 1.6338 - val_accuracy: 0.4292 - lr: 6.7032e-04
Epoch 6/10
23090/23090 [==============================] - 28s 1ms/step - loss: 1.6917 - accuracy: 0.4102 - val_loss: 1.6309 - val_accuracy: 0.4297 - lr: 6.0653e-04
Epoch 7/10
23090/230

In [48]:
model.save("C:\\Users\\abubabu\\Documents\\GitHub\\synthetic-population-simulation\\Model\\v2.h5")